In [3]:
import requests
from bs4 import BeautifulSoup
import selenium

from selenium.webdriver import Firefox
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, WebDriverException

from time import sleep

/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a

In [65]:
https = "https://elibrary.ru"
artical_data = {}

In [8]:
r = requests.get(https + '/title_items.asp?id=21814')
soup = BeautifulSoup(r.content.decode('utf-8'), "html.parser")

In [ ]:
browser = Firefox(executable_path="./geckodriver") # ./geckodriver тоже вроде можно
browser.get('https://elibrary.ru/title_items.asp?id=21814')
# for i in range(3):
page = browser.page_source
artical_adresses = get_articles_adress(page)
for adress in articles_adrees:
    d = find_data(adress)
    artical_data[d[1]] = [d[0], d[2]]
xp = '/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'
search_form = browser.find_element_by_xpath(xp)
search_form.send_keys(Keys.ENTER)
print(artical_data)

In [119]:
def get_articles_adress(page):
    '''
    Возвращает список адресов всех статей
    '''
    articles_adrees = []
    id_str = '/item.asp?id='
    for i in page.findAll('a'):
        if id_str in i['href']:
            num_id = i['href']
            articles_adrees.append(https + num_id)
    return articles_adrees

def find_data(adress):
    '''
    Возвращает list из названия [статьи, авторы, абстракт]
    '''
    assert isinstance(adress, str)
    art = requests.get(adress)
    soup = BeautifulSoup(art.content.decode('utf-8'), "html.parser")
    data = []
    for i in soup.findAll('input'):
        try:    
            if not i['value'][0].isdigit() and (i['value'][0] == i['value'][0].upper()):
                data.append(i['value'])
        except:
            continue
    data.append(soup.find('p').text)
    return data
    

In [116]:
artical_data = {}
for adress in articles_adrees:
    d = find_data(adress)
    artical_data[d[1]] = [d[0], d[2]]

In [117]:
artical_data

{'Dual view sliding phases symmetric systems': ['Вайл С|Куклов А Б|Оганесян В|',
  ''],
 'Origin stationary domain wall enhanced ferroelectric susceptibility': ['Лю С|Цохен Р Е|',
  ''],
 'Understanding evolution anomalous anharmonicity Bi2Te3-xSex': ['Бурч К С|Гу Г|Жонг Р|Йия С|Тьян Ю|Цава Р Ж|Щнеелоч Ж|',
  ''],
 'Zn vacancy polaronic hole trap ZnO': ['Алкаускас А|Берхеим Т С|Ехансен К М|Свенсон Б Г|Фродасон Ю К|',
  ''],
 'Classical quantum thermodynamics lattice model exhibiting structural negative thermal expansion': ['Оцчялини Ц А|Хандунканда С У|Ханцок Ж Н|Цурры Е Б|',
  ''],
 'Effects biaxial strain improper multiferroicity h-LuFe O3 films studied using restrained thermal expansion method': ['Бовлан Ж|Ванг Х|Ву Х|Дичяра А Д|Жанг Х|Жанг Ю|Йянг Х|Ким Ж В|Ксу Х|Ли Ю|Рьян П Ж|Синха К|Ченг Х|Яроцкий Д А|',
  ''],
 'Dielectric properties amorphous phase-change materials': ['Вирцсохн М|Волкер Х|Вуттиг М|Енгельманн У|Ест П|Каминский М|Кругер К|Лобо Р П С М|Чен Ц|Щлич Ф|Щлокерманн Ц|',


In [7]:
browser = Firefox(executable_path="./geckodriver") # ./geckodriver тоже вроде можно
browser.get('https://elibrary.ru/title_items.asp?id=21814')
xp = '/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'
search_form = browser.find_element_by_xpath(xp)
search_form.send_keys(Keys.ENTER)

WebDriverException: Message: TypeError: this.curBrowser.contentBrowser is null


In [2]:
html = browser.page_source
print(html)

NameError: name 'browser' is not defined